In [107]:
# For now, clear outputs (Cell > All Output > Clear) before committing to Git
# There might be a better way

import sqlite3
import pandas as pd
cnx = sqlite3.connect('FPA_FOD_20170508.sqlite')

import matplotlib.pyplot as plt
import numpy as np
from sklearn import tree, preprocessing
import sklearn.ensemble as ske
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [127]:
df = pd.read_sql_query("SELECT FIRE_YEAR,DISCOVERY_TIME,STAT_CAUSE_DESCR,CONT_DATE,CONT_TIME,LATITUDE,LONGITUDE,STATE,DISCOVERY_DATE,FIRE_SIZE,FIRE_SIZE_CLASS FROM 'Fires'", cnx)
print(df.head())

   FIRE_YEAR DISCOVERY_TIME STAT_CAUSE_DESCR  CONT_DATE CONT_TIME   LATITUDE  \
0       2005           1300    Miscellaneous  2453403.5      1730  40.036944   
1       2004           0845        Lightning  2453137.5      1530  38.933056   
2       2004           1921   Debris Burning  2453156.5      2024  38.984167   
3       2004           1600        Lightning  2453189.5      1400  38.559167   
4       2004           1600        Lightning  2453189.5      1200  38.559167   

    LONGITUDE STATE  DISCOVERY_DATE  FIRE_SIZE FIRE_SIZE_CLASS  
0 -121.005833    CA       2453403.5       0.10               A  
1 -120.404444    CA       2453137.5       0.25               A  
2 -120.735556    CA       2453156.5       0.10               A  
3 -119.913333    CA       2453184.5       0.10               A  
4 -119.933056    CA       2453184.5       0.10               A  


In [128]:
df['DISCOVERY_DATE'] = pd.to_datetime(df['DISCOVERY_DATE'] - pd.Timestamp(0).to_julian_date(), unit='D')
df['CONT_DATE'] = pd.to_datetime(df['CONT_DATE'] - pd.Timestamp(0).to_julian_date(), unit='D')
print(df.head())

   FIRE_YEAR DISCOVERY_TIME STAT_CAUSE_DESCR  CONT_DATE CONT_TIME   LATITUDE  \
0       2005           1300    Miscellaneous 2005-02-02      1730  40.036944   
1       2004           0845        Lightning 2004-05-12      1530  38.933056   
2       2004           1921   Debris Burning 2004-05-31      2024  38.984167   
3       2004           1600        Lightning 2004-07-03      1400  38.559167   
4       2004           1600        Lightning 2004-07-03      1200  38.559167   

    LONGITUDE STATE DISCOVERY_DATE  FIRE_SIZE FIRE_SIZE_CLASS  
0 -121.005833    CA     2005-02-02       0.10               A  
1 -120.404444    CA     2004-05-12       0.25               A  
2 -120.735556    CA     2004-05-31       0.10               A  
3 -119.913333    CA     2004-06-28       0.10               A  
4 -119.933056    CA     2004-06-28       0.10               A  


In [134]:
df['DISCOVERY_DATE']

0         2005-02-02
1         2004-05-12
2         2004-05-31
3         2004-06-28
4         2004-06-28
5         2004-06-30
6         2004-07-01
7         2005-03-08
8         2005-03-15
9         2004-07-01
10        2004-07-02
11        2004-07-02
12        2004-09-03
13        2004-09-28
14        2004-10-03
15        2004-10-03
16        2004-10-06
17        2004-10-13
18        2004-11-20
19        2004-06-04
20        2004-06-19
21        2004-06-21
22        2004-06-25
23        2004-07-01
24        2004-07-01
25        2004-07-20
26        2004-03-18
27        2004-08-05
28        2004-03-18
29        2004-05-11
             ...    
1880435   2015-06-28
1880436   2015-03-21
1880437   2015-05-11
1880438   2015-09-26
1880439   2015-01-27
1880440   2015-07-04
1880441   2015-12-07
1880442   2015-03-28
1880443   2015-09-27
1880444   2015-10-30
1880445   2015-02-22
1880446   2015-07-25
1880447   2015-01-28
1880448   2015-06-26
1880449   2015-08-27
1880450   2015-06-27
1880451   201

In [135]:
df['CONT_TIME']

0          1730
1          1530
2          2024
3          1400
4          1200
5          1600
6          1400
7          1600
8          1700
9          1800
10         1900
11         2000
12         1630
13         1800
14         1500
15         1400
16         1000
17         1800
18         1630
19         1400
20         1940
21         1100
22         1300
23         1110
24         1450
25         1600
26         2030
27         1815
28         2130
29         1552
           ... 
1880435    None
1880436    1152
1880437    2000
1880438    1650
1880439    None
1880440    None
1880441    None
1880442    1750
1880443    0855
1880444    None
1880445    None
1880446    None
1880447    None
1880448    2014
1880449    None
1880450    None
1880451    1228
1880452    None
1880453    None
1880454    None
1880455    0000
1880456    1913
1880457    0056
1880458    1000
1880459    1436
1880460    1843
1880461    None
1880462    None
1880463    None
1880464    None
Name: CONT_TIME, Length:

In [133]:
df['CONT_DATE']

0         2005-02-02
1         2004-05-12
2         2004-05-31
3         2004-07-03
4         2004-07-03
5         2004-07-01
6         2004-07-02
7         2005-03-08
8         2005-03-15
9         2004-07-02
10        2004-07-03
11        2004-07-03
12        2004-09-03
13        2004-09-28
14        2004-10-03
15        2004-10-03
16        2004-10-21
17        2004-10-17
18        2004-11-21
19        2004-06-04
20        2004-06-19
21        2004-06-22
22        2004-06-26
23        2004-07-02
24        2004-07-06
25        2004-07-20
26        2004-03-18
27        2004-08-05
28        2004-03-18
29        2004-05-11
             ...    
1880435          NaT
1880436   2015-03-21
1880437   2015-05-11
1880438   2015-09-26
1880439          NaT
1880440          NaT
1880441          NaT
1880442   2015-03-28
1880443   2015-09-27
1880444          NaT
1880445          NaT
1880446          NaT
1880447          NaT
1880448   2015-06-26
1880449          NaT
1880450          NaT
1880451   201

In [120]:
df['DISCOVERY_MONTH'] = pd.DatetimeIndex(df['DISCOVERY_DATE']).month
df['DISCOVERY_DAY'] = pd.DatetimeIndex(df['DISCOVERY_DATE']).day
df['DISCOVERY_DAY_OF_WEEK'] = df['DISCOVERY_DATE'].dt.weekday_name

df['CONT_MONTH'] = pd.DatetimeIndex(df['CONT_DATE']).month
df['CONT_DAY'] = pd.DatetimeIndex(df['CONT_DATE']).day
df['CONT_DAY_OF_WEEK'] = df['CONT_DATE'].dt.weekday_name
print(df.head())

   FIRE_YEAR DISCOVERY_TIME STAT_CAUSE_DESCR  CONT_DATE CONT_TIME   LATITUDE  \
0       2005           1300    Miscellaneous 2005-02-02      1730  40.036944   
1       2004           0845        Lightning 2004-05-12      1530  38.933056   
2       2004           1921   Debris Burning 2004-05-31      2024  38.984167   
3       2004           1600        Lightning 2004-07-03      1400  38.559167   
4       2004           1600        Lightning 2004-07-03      1200  38.559167   

    LONGITUDE STATE DISCOVERY_DATE  FIRE_SIZE FIRE_SIZE_CLASS  \
0 -121.005833    CA     2005-02-02       0.10               A   
1 -120.404444    CA     2004-05-12       0.25               A   
2 -120.735556    CA     2004-05-31       0.10               A   
3 -119.913333    CA     2004-06-28       0.10               A   
4 -119.933056    CA     2004-06-28       0.10               A   

   DISCOVERY_MONTH  DISCOVERY_DAY DISCOVERY_DAY_OF_WEEK  CONT_MONTH  CONT_DAY  \
0                2              2             W

In [122]:
le = preprocessing.LabelEncoder()
df['STAT_CAUSE_DESCR'] = le.fit_transform(df['STAT_CAUSE_DESCR'])
df['STATE'] = le.fit_transform(df['STATE'])
df['DISCOVERY_DAY_OF_WEEK'] = le.fit_transform(df['DISCOVERY_DAY_OF_WEEK'])
#df['CONT_DAY_OF_WEEK'] = le.fit_transform(df['CONT_DAY_OF_WEEK'])
print(df.head())

   FIRE_YEAR DISCOVERY_TIME  STAT_CAUSE_DESCR  CONT_DATE CONT_TIME   LATITUDE  \
0       2005           1300                 7 2005-02-02      1730  40.036944   
1       2004           0845                 6 2004-05-12      1530  38.933056   
2       2004           1921                 3 2004-05-31      2024  38.984167   
3       2004           1600                 6 2004-07-03      1400  38.559167   
4       2004           1600                 6 2004-07-03      1200  38.559167   

    LONGITUDE  STATE DISCOVERY_DATE  FIRE_SIZE FIRE_SIZE_CLASS  \
0 -121.005833      4     2005-02-02       0.10               A   
1 -120.404444      4     2004-05-12       0.25               A   
2 -120.735556      4     2004-05-31       0.10               A   
3 -119.913333      4     2004-06-28       0.10               A   
4 -119.933056      4     2004-06-28       0.10               A   

   DISCOVERY_MONTH  DISCOVERY_DAY  DISCOVERY_DAY_OF_WEEK  CONT_MONTH  \
0                2              2           

In [124]:
df['CONT_DAY_OF_WEEK']

0          Wednesday
1          Wednesday
2             Monday
3           Saturday
4           Saturday
5           Thursday
6             Friday
7            Tuesday
8            Tuesday
9             Friday
10          Saturday
11          Saturday
12            Friday
13           Tuesday
14            Sunday
15            Sunday
16          Thursday
17            Sunday
18            Sunday
19            Friday
20          Saturday
21           Tuesday
22          Saturday
23            Friday
24           Tuesday
25           Tuesday
26          Thursday
27          Thursday
28          Thursday
29           Tuesday
             ...    
1880435          NaN
1880436     Saturday
1880437       Monday
1880438     Saturday
1880439          NaN
1880440          NaN
1880441          NaN
1880442     Saturday
1880443       Sunday
1880444          NaN
1880445          NaN
1880446          NaN
1880447          NaN
1880448       Friday
1880449          NaN
1880450          NaN
1880451     S

In [ ]:
logits=df.drop(['FIRE_SIZE','DATE','DISCOVERY_DATE'],axis=1)

In [ ]:
labels=df['FIRE_SIZE']
labels.head()

In [ ]:
logits=df.drop(['FIRE_SIZE','DATE','DISCOVERY_DATE'],axis=1)
logits.head()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(logits,labels,test_size=0.2)

In [ ]:
reg = LinearRegression().fit(x_train, y_train)

In [ ]:
reg.score(x_test,y_test)

In [ ]:
import tensorflow
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation

In [ ]:
from keras.activations import relu

In [ ]:
model=Sequential()
model.add(Dense(50,input_shape=[8]))
model.add(Activation('relu'))
model.add(Dense(1))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='sgd',
              loss='mean_squared_error',
              metrics=['accuracy'])

In [ ]:
model.fit(x_train,y_train,epochs=10)

In [ ]:
model.evaluate(x_test,y_test)

In [ ]:
a=model.predict(x_test)

In [ ]:
a[100]

In [ ]:
df['STAT_CAUSE_DESCR'].value_counts().plot(kind='barh',color='coral')
plt.show()

In [ ]:
df_arson = df[df['STAT_CAUSE_DESCR']=='Arson']
df_arson['DAY_OF_WEEK'].value_counts().plot(kind='barh',color='coral')
plt.show()

In [ ]:
def plot_corr(df,size=10):
    corr = df.corr()  #the default method is pearson
    fig, ax = plt.subplots(figsize=(size, size))
    ax.matshow(corr,cmap=plt.cm.Oranges)
    plt.xticks(range(len(corr.columns)), corr.columns)
    plt.yticks(range(len(corr.columns)), corr.columns)
    for tick in ax.get_xticklabels():
        tick.set_rotation(45)    
    plt.show()
    

    
plot_corr(df)

In [ ]:
df.plot(kind='scatter',x='LONGITUDE',y='LATITUDE',color='coral',alpha=0.3)
plt.show()

In [ ]:
df_lightning = df[df['STAT_CAUSE_DESCR']=='Lightning']
df_lightning['DAY_OF_WEEK'].value_counts().plot(kind='barh',color='coral')
plt.show()

In [ ]:
df['DAY_OF_WEEK'].value_counts().plot(kind='barh',color='coral')
plt.show()

In [ ]:
df['STATE'].value_counts().head(n=10).plot(kind='barh',color='coral')
plt.show()

In [ ]:
df_CA = df[df['STATE']=='CA']
df_CA['STAT_CAUSE_DESCR'].value_counts().plot(kind='barh',color='coral',title='causes of fires for CA')
plt.show()